In [6]:
with open("import_modules.py") as f:
    exec(f.read())

ModuleNotFoundError: No module named 'pydot'

In [2]:
df1 = pd.read_csv('books_new_modified_1348.csv')
df2 = pd.read_csv('wiki_extended_divided_44654.csv')
df3 = pd.read_csv('essays_ext_3828.csv')
df_tot = pd.concat((df1, df2, df3), ignore_index= True)

In [3]:
url = 'http://localhost:8000/getExtraction'
headers = {'Content-type': 'text/plain'}   

In [4]:
roman_reg = re.compile('(?<=^)M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})(?=$)')

def romanToInt(s):
    translations = {
        "I": 1,
        "V": 5,
        "X": 10,
        "L": 50,
        "C": 100,
        "D": 500,
        "M": 1000
    }
    number = 0
    s = s.replace("IV", "IIII").replace("IX", "VIIII")
    s = s.replace("XL", "XXXX").replace("XC", "LXXXX")
    s = s.replace("CD", "CCCC").replace("CM", "DCCCC")
    for char in s:
        number += translations[char]
    return number

def get_matches(text):
    tokenize = nltk.word_tokenize(text)
    dict = {}
    for token in tokenize:
        if roman_reg.match(token):
            dict[token] = romanToInt(token)
    return dict

def replace_matches(text):
    matches = get_matches(text)
    for key in matches:
        text = text.replace(key, str(matches[key]))
    return text

In [ ]:
#Pipeline with spacy to substitute pronouns 
df = pd.read_csv('new_example.csv')
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)
d = {}
for i in range(len(df)):
    doc = nlp(df['text'][i])
    d[df['title'][i]] = doc._.coref_resolved
new_df = pd.DataFrame(list(d.items()),columns = ['title','text'])

In [5]:
def sentence_tokenization(df):
    out = [sent_tokenize(par) for par in df.text]
    final=[]
    for par in out:
        clean = []
        for i in range(len(par)):
            if len(par[i]) > 5:
                clean.append(par[i])
        final.append(clean)
    return final

def relation_extraction(out):
    issues = 0
    #out = sentence_tokenization(df)
    rel_ext = []
    for par in out:
        for s in par[:100]:
            s = unidecode(s.replace(' ', ' ').replace('/', ' '))
            s = s.replace('$', '').replace('. . .', '. ').replace('...', '. ')
            s = replace_matches(s)
            s = s.replace(' the ', ' ')
            req = requests.request('POST', url, headers=headers, data = s.encode('utf-8'))
            try:
                js = req.json()
                rel_ext.extend(js)
            except:
                issues +=1
                #print('JSON issue')
    return issues, rel_ext

def create_rel_dataframe(rel):
    #issues, rel = relation_extraction(df)
    l  = []
    for r in rel:
        try:
            l.append((r['confidence'], r['sentence'], r['extraction']['arg1']['text'], r['extraction']['rel']['text'], 
                    r['extraction']['arg2s'][0]['text'], r['extraction']['negated'], r['extraction']['passive']))
        except IndexError:
            pass 
    
    l = list(dict.fromkeys(l))
    return pd.DataFrame(l, columns= ['confidence', 'sentence', 'arg1', 'rel', 'arg2', 'negated', 'passive'])

In [9]:
import random
out = sentence_tokenization(df_tot)
random.shuffle(out)

In [ ]:
issues, rel = relation_extraction(out)
print(f'N. Issues: {issues}')

In [ ]:
rel_df = create_rel_dataframe(rel)

In [ ]:
rel_df = rel_df[rel_df['arg2'].apply(lambda x: len(x.split(' ')) > 1)].reset_index(drop = True)
rel_df = rel_df[rel_df['rel'].apply(lambda x: len(x.split(' ')) < 6)].reset_index(drop = True)